In [1]:
import os
import sys
import pathlib
import json
import requests
from collections import deque

import geopy as gp

import pandas as pd
import numpy as np

import overpass

import seaborn as sns
import matplotlib as plt

import openrouteservice as ors
from shapely.geometry import Point, LineString

import warnings
warnings.filterwarnings('ignore')

project_path = pathlib.Path.cwd().parent.parent

In [2]:
project_path = pathlib.Path.cwd().parent.parent

In [3]:
town_halls_coor = pd.read_excel(str(project_path) + "/data/Coordinates/coor.xlsx")

In [4]:
min_lat = town_halls_coor.lat.min()
max_lat = town_halls_coor.lat.max()
min_lon = town_halls_coor.lon.min()
max_lon = town_halls_coor.lon.max()

In [5]:
padding = 0.01
n_bound = max_lat + padding
s_bound = min_lat - padding
e_bound = max_lon + padding
w_bound = min_lon - padding

In [6]:
print(f"{n_bound:.4f}, {s_bound:.4f}, {e_bound:.4f}, {w_bound:.4f}")

19.9307, 6.1801, 104.8143, 97.9545


In [7]:
bbox = (n_bound, s_bound, e_bound, w_bound)

In [8]:
overpass_query = f"""
[out:json][timeout:60];
(
  way["highway"="motorway"]{bbox};  
  way["highway"="trunk"]{bbox};
  way["highway"="primary"]{bbox};
);
out geom;
"""

In [10]:
try:
    api = overpass.API()
    response = api.get(overpass_query, responseformat="geojson")
    
    if response and response.get("features"):
        feature_count = len(response["features"])
    
        output_filename = str(pathlib.Path.cwd().parent.parent) + r"\data\road_network\bbox_major_highways.geojson"
        
        with open(output_filename, "w", encoding="utf-8") as f_out:
            json.dump(response, f_out, ensure_ascii=False, indent=2)
        
        print(f"Data has been saved to '{output_filename}")
    else:
        print("The query returned no features. The area might not contain major highways.")
    
except overpass.errors.OverpassError as e:
    print(f"An error occured due to the Overpass API: {e}")
except Exception as e:
    print(f"An unexpected error occured: {e}")

An error occured due to the Overpass API: [out:json];
[out:json][timeout:60];
(
  way["highway"="motorway"](np.float64(19.930675), np.float64(6.1800880000000005), np.float64(104.8142845), np.float64(97.9545338));  
  way["highway"="trunk"](np.float64(19.930675), np.float64(6.1800880000000005), np.float64(104.8142845), np.float64(97.9545338));
  way["highway"="primary"](np.float64(19.930675), np.float64(6.1800880000000005), np.float64(104.8142845), np.float64(97.9545338));
);
out geom;out body;
